In [1]:
import pandas as pd
import math
import numpy as np

%matplotlib inline

from pathlib import Path
from omegaconf import OmegaConf
import scipy.stats as st

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "valid_log.csv").is_file():
        cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
        cfg = OmegaConf.to_container(cfg)
        num_epochs = cfg['optim']['epochs']

        valid_log = pd.read_csv(run / "valid_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-3) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("valid_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    len(list(Path('.').glob('*')))
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)

    run_number = cfg['data']['train']['cross_val_bucket_validation_index']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / 'test_predictions' / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = run_number
    data['inv_temp'] = inv_temp
    data['regime'] = regime
    
    return data

def collect_all(model_name, root, csv_file):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            for run in list(regime.glob("run-*")):
                metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

def collect_all_regimes(model_name, root, csv_file, regimes=['0.1', '0.2', '0.5', '1.0']):
    root = Path(root)

    metrics = []
    for run in list(root.glob("run-*")):
        if root.name.rsplit("-", 1)[1] in regimes:
            metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [4]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], model_group[1], model_group[2], model_group[3]
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], t_student=False):
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate(['mean', 'std'])
    
    return mean_metrics

<h1>GlaS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [5]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t')),
}

In [6]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [7]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,1.0,1.0,52.31,36.16,41.72,8.55
1,H-UNet-BASE-SWTA,0,5.0,1.0,49.74,33.87,42.40,8.16
2,H-UNet-BASE-SWTA,0,10.0,1.0,45.79,30.24,45.59,8.88
3,H-UNet-BASE-SWTA,0,20.0,1.0,49.81,33.74,38.32,7.20
4,H-UNet-BASE-SWTA,0,50.0,1.0,49.98,34.01,42.11,7.88
5,H-UNet-BASE-SWTA,0,100.0,1.0,49.48,33.46,40.05,7.27
6,H-UNet-BASE-SWTA-T,0,1.0,1.0,45.60,30.22,44.54,9.98
7,H-UNet-BASE-SWTA-T,0,5.0,1.0,48.07,32.26,54.12,10.52
8,H-UNet-BASE-SWTA-T,0,10.0,1.0,49.61,33.60,42.02,8.29
9,H-UNet-BASE-SWTA-T,0,20.0,1.0,50.11,34.03,42.04,7.84


Dice     Jaccard     Hausdorff Distance  \
                                     mean std    mean std               mean   
Model              Inv Temp Regime                                             
H-UNet-BASE-SWTA   1.0      1.0     52.31 NaN   36.16 NaN              41.72   
                   5.0      1.0     49.74 NaN   33.87 NaN              42.40   
                   10.0     1.0     45.79 NaN   30.24 NaN              45.59   
                   20.0     1.0     49.81 NaN   33.74 NaN              38.32   
                   50.0     1.0     49.98 NaN   34.01 NaN              42.11   
                   100.0    1.0     49.48 NaN   33.46 NaN              40.05   
H-UNet-BASE-SWTA-T 1.0      1.0     45.60 NaN   30.22 NaN              44.54   
                   5.0      1.0     48.07 NaN   32.26 NaN              54.12   
                   10.0     1.0     49.61 NaN   33.60 NaN              42.02   
                   20.0     1.0     50.11 NaN   34.03 NaN              42.04   
                   50.0     1.0     47.67 NaN   31.89 NaN              43.38   
                   100.0    1.0     49.24 NaN   33.43 NaN              41.65   

                                       Average Surface Distance      
                                   std                     mean std  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   1.0      1.0    NaN                     8.55 NaN  
                   5.0      1.0    NaN                     8.16 NaN  
                   10.0     1.0    NaN                     8.88 NaN  
                   20.0     1.0    NaN                     7.20 NaN  
                   50.0     1.0    NaN                     7.88 NaN  
                   100.0    1.0    NaN                     7.27 NaN  
H-UNet-BASE-SWTA-T 1.0      1.0    NaN                     9.98 NaN  
                   5.0      1.0    NaN                    10.52 NaN  
                   10.0     1.0    NaN                     8.29 NaN  
                   20.0     1.0    NaN                     7.84 NaN  
                   50.0     1.0    NaN                     8.20 NaN  
                   100.0    1.0    NaN                     8.29 NaN

In [8]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    #'H-UNet-BASE-SWTA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft_fromBestDice')),
    #'H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft_fromBestDice')),
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft')),
    'H-UNet-Pseudolabeling-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t_ft')),
}

In [9]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [10]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-FT,0,1.0,0.2,78.29,65.23,25.62,3.60
1,H-UNet-BASE-SWTA-FT,0,5.0,0.2,79.96,67.77,27.76,3.96
2,H-UNet-BASE-SWTA-FT,0,10.0,0.2,81.71,70.05,23.32,3.45
3,H-UNet-BASE-SWTA-FT,0,20.0,0.2,81.64,69.71,19.47,2.40
4,H-UNet-BASE-SWTA-FT,0,50.0,0.2,81.50,69.69,22.19,3.04
...,...,...,...,...,...,...,...,...
115,H-UNet-BASE-SWTA-T-FT,9,5.0,0.2,80.07,68.01,22.94,3.58
116,H-UNet-BASE-SWTA-T-FT,9,10.0,0.2,79.62,67.11,26.51,3.54
117,H-UNet-BASE-SWTA-T-FT,9,20.0,0.2,80.61,68.63,28.82,3.85
118,H-UNet-BASE-SWTA-T-FT,9,50.0,0.2,79.22,66.88,27.03,3.87


Dice           Jaccard            \
                                         mean       std    mean       std   
Model                 Inv Temp Regime                                       
H-UNet-BASE-SWTA-FT   1.0      0.2     77.890  1.997649  64.949  2.580833   
                      5.0      0.2     79.583  1.966842  67.351  2.601664   
                      10.0     0.2     79.743  1.593606  67.467  2.118732   
                      20.0     0.2     80.649  1.280663  68.642  1.708799   
                      50.0     0.2     80.322  1.563833  68.282  2.008149   
                      100.0    0.2     80.417  2.325621  68.408  3.034636   
H-UNet-BASE-SWTA-T-FT 1.0      0.2     78.210  1.914233  65.403  2.549466   
                      5.0      0.2     80.459  1.786520  68.528  2.441210   
                      10.0     0.2     81.042  1.461079  69.245  1.992058   
                      20.0     0.2     81.511  2.092566  69.898  2.830684   
                      50.0     0.2     81.150  1.887467  69.536  2.490543   
                      100.0    0.2     80.874  2.195269  69.032  2.904953   

                                      Hausdorff Distance            \
                                                    mean       std   
Model                 Inv Temp Regime                                
H-UNet-BASE-SWTA-FT   1.0      0.2                28.680  3.489909   
                      5.0      0.2                26.779  5.198413   
                      10.0     0.2                26.086  5.496421   
                      20.0     0.2                24.566  4.580646   
                      50.0     0.2                25.608  2.128227   
                      100.0    0.2                25.240  4.685384   
H-UNet-BASE-SWTA-T-FT 1.0      0.2                28.162  5.176704   
                      5.0      0.2                25.713  3.538424   
                      10.0     0.2                24.659  4.680369   
                      20.0     0.2                25.000  4.665269   
                      50.0     0.2                24.798  3.130129   
                      100.0    0.2                25.238  5.418556   

                                      Average Surface Distance            
                                                          mean       std  
Model                 Inv Temp Regime                                     
H-UNet-BASE-SWTA-FT   1.0      0.2                       4.102  0.595871  
                      5.0      0.2                       3.807  0.716505  
                      10.0     0.2                       3.717  0.720140  
                      20.0     0.2                       3.346  0.760675  
                      50.0     0.2                       3.521  0.304939  
                      100.0    0.2                       3.545  0.652521  
H-UNet-BASE-SWTA-T-FT 1.0      0.2                       4.079  0.975676  
                      5.0      0.2                       3.632  0.391828  
                      10.0     0.2                       3.455  0.584964  
                      20.0     0.2                       3.445  0.671669  
                      50.0     0.2                       3.545  0.526566  
                      100.0    0.2                       3.458  0.742949

<h2>Evaluation - Hebbian Unsupervised Pretraining</h2>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [11]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

INV_TEMP_GlaS=20          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
}

In [12]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [13]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

/tmp/ipykernel_1874990/265248669.py:24: RuntimeWarning: Mean of empty slice
  mean_value = np.nanmean(values)


,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,20.0,1.0,49.81,33.74,38.32,7.20
1,H-UNet-BASE-SWTA-T,0,20.0,1.0,50.11,34.03,42.04,7.84
2,H-UNet-HPCA,0,1.0,1.0,33.72,20.47,NaN,NaN
3,H-UNet-HPCA-T,0,1.0,1.0,34.84,21.30,NaN,NaN


Dice     Jaccard     Hausdorff Distance  \
                                     mean std    mean std               mean   
Model              Inv Temp Regime                                             
H-UNet-BASE-SWTA   20.0     1.0     49.81 NaN   33.74 NaN              38.32   
H-UNet-BASE-SWTA-T 20.0     1.0     50.11 NaN   34.03 NaN              42.04   
H-UNet-HPCA        1.0      1.0     33.72 NaN   20.47 NaN                NaN   
H-UNet-HPCA-T      1.0      1.0     34.84 NaN   21.30 NaN                NaN   

                                       Average Surface Distance      
                                   std                     mean std  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   20.0     1.0    NaN                     7.20 NaN  
H-UNet-BASE-SWTA-T 20.0     1.0    NaN                     7.84 NaN  
H-UNet-HPCA        1.0      1.0    NaN                      NaN NaN  
H-UNet-HPCA-T      1.0      1.0    NaN                      NaN NaN

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

In [14]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

REGIMES = ['0.05', '0.1' '0.2', '0.25']       # regimes to be considered
INV_TEMP_GlaS=20          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=glas/unet_base/inv_temp-1').glob('regime-*')),
    #'UNet-256': list(Path(EXP_ROOT + '/experiment=glas/unet_base-256/inv_temp-1').glob('regime-*')),
    'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=glas/unet/inv_temp-1').glob('regime-*')),
    #'H-UNet-HPCA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_ft_fromBestDice/inv_temp-1').glob('regime-*')),
    #'H-UNet-HPCA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t_ft_fromBestDice/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    #'H-UNet-SWTA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft_fromBestDice/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    #'H-UNet-SWTA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft_fromBestDice/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
}

In [15]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all_regimes(k, r, 'preds.csv', regimes=REGIMES) for k, v in runs.items() for r in v], ignore_index=True)

In [16]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-HPCA-FT,0,1.0,0.2,78.65,65.73,26.27,3.62
1,H-UNet-HPCA-FT,1,1.0,0.2,79.22,66.37,28.45,3.44
2,H-UNet-HPCA-FT,2,1.0,0.2,76.39,62.88,23.08,3.02
3,H-UNet-HPCA-FT,3,1.0,0.2,80.71,68.70,25.28,3.41
4,H-UNet-HPCA-FT,4,1.0,0.2,79.80,67.31,21.33,3.07
...,...,...,...,...,...,...,...,...
75,UNet-Pseudolabeling,5,1.0,0.2,81.56,70.00,40.64,5.20
76,UNet-Pseudolabeling,6,1.0,0.2,80.22,68.10,29.06,3.93
77,UNet-Pseudolabeling,7,1.0,0.2,81.88,70.81,35.93,4.94
78,UNet-Pseudolabeling,8,1.0,0.2,82.94,72.35,25.41,4.34


Dice           Jaccard  \
                                                   mean       std    mean   
Model                           Inv Temp Regime                             
H-UNet-HPCA-FT                  1.0      0.2     79.593  1.425997  67.142   
H-UNet-HPCA-T-FT                1.0      0.2     79.974  1.944258  67.762   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.2     76.306  5.248513  63.337   
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.2     76.829  4.794669  64.225   
H-UNet-SWTA-FT                  20.0     0.2     80.649  1.280663  68.642   
H-UNet-SWTA-T-FT                20.0     0.2     81.511  2.092566  69.898   
UNet                            1.0      0.2     79.676  2.017701  67.314   
UNet-Pseudolabeling             1.0      0.2     81.449  2.045317  70.058   

                                                          Hausdorff Distance  \
                                                      std               mean   
Model                           Inv Temp Regime                                
H-UNet-HPCA-FT                  1.0      0.2     1.964755             25.142   
H-UNet-HPCA-T-FT                1.0      0.2     2.697887             26.488   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.2     6.499253             36.581   
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.2     5.799692             38.882   
H-UNet-SWTA-FT                  20.0     0.2     1.708799             24.566   
H-UNet-SWTA-T-FT                20.0     0.2     2.830684             25.000   
UNet                            1.0      0.2     2.674805             26.129   
UNet-Pseudolabeling             1.0      0.2     2.743845             31.266   

                                                            \
                                                       std   
Model                           Inv Temp Regime              
H-UNet-HPCA-FT                  1.0      0.2      4.265947   
H-UNet-HPCA-T-FT                1.0      0.2      5.306534   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.2      9.370050   
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.2     10.157618   
H-UNet-SWTA-FT                  20.0     0.2      4.580646   
H-UNet-SWTA-T-FT                20.0     0.2      4.665269   
UNet                            1.0      0.2      4.898783   
UNet-Pseudolabeling             1.0      0.2      5.851114   

                                                Average Surface Distance  \
                                                                    mean   
Model                           Inv Temp Regime                            
H-UNet-HPCA-FT                  1.0      0.2                       3.466   
H-UNet-HPCA-T-FT                1.0      0.2                       3.581   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.2                       5.420   
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.2                       5.900   
H-UNet-SWTA-FT                  20.0     0.2                       3.346   
H-UNet-SWTA-T-FT                20.0     0.2                       3.445   
UNet                            1.0      0.2                       3.541   
UNet-Pseudolabeling             1.0      0.2                       4.418   

                                                           
                                                      std  
Model                           Inv Temp Regime            
H-UNet-HPCA-FT                  1.0      0.2     0.615904  
H-UNet-HPCA-T-FT                1.0      0.2     0.726735  
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.2     1.443306  
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.2     1.713223  
H-UNet-SWTA-FT                  20.0     0.2     0.760675  
H-UNet-SWTA-T-FT                20.0     0.2     0.671669  
UNet                            1.0      0.2     0.786617  
UNet-Pseudolabeling             1.0      0.2     0.738042

<h1>PH2 Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [19]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_t')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

In [ ]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_t_ft')),
    'H-UNet-Pseudolabeling-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta_t')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta_t_ft')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

<h2>Evaluation - Hebbian Unsupervised Pretraining</h2>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [21]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

INV_TEMP_PH2=20          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run-*')),
}

In [22]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [23]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

/tmp/ipykernel_1874990/265248669.py:24: RuntimeWarning: Mean of empty slice
  mean_value = np.nanmean(values)


,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-HPCA,0,1.0,1.0,21.89,12.59,104.8,41.46
1,H-UNet-HPCA-T,0,1.0,1.0,21.21,12.13,NaN,NaN


Dice     Jaccard     Hausdorff Distance      \
                                mean std    mean std               mean std   
Model         Inv Temp Regime                                                 
H-UNet-HPCA   1.0      1.0     21.89 NaN   12.59 NaN              104.8 NaN   
H-UNet-HPCA-T 1.0      1.0     21.21 NaN   12.13 NaN                NaN NaN   

                              Average Surface Distance      
                                                  mean std  
Model         Inv Temp Regime                               
H-UNet-HPCA   1.0      1.0                       41.46 NaN  
H-UNet-HPCA-T 1.0      1.0                         NaN NaN

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

In [24]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

REGIMES = ['0.05', '0.1' '0.2', '0.25']       # regimes to be considered
INV_TEMP_PH2=10          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=ph2/unet_base/inv_temp-1').glob('regime-*')),
    'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=ph2/unet/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-hpca_t_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
}

In [25]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all_regimes(k, r, 'preds.csv', regimes=REGIMES) for k, v in runs.items() for r in v], ignore_index=True)

In [26]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-HPCA-FT,0,1.0,0.2,87.42,78.72,9.53,1.31
1,H-UNet-HPCA-FT,1,1.0,0.2,85.41,75.90,12.21,2.03
2,H-UNet-HPCA-FT,2,1.0,0.2,90.04,82.07,1.74,0.27
3,H-UNet-HPCA-FT,3,1.0,0.2,89.56,81.60,3.04,0.42
4,H-UNet-HPCA-FT,4,1.0,0.2,88.13,80.06,9.58,1.35
5,H-UNet-HPCA-FT,5,1.0,0.2,87.63,78.99,5.08,0.78
6,H-UNet-HPCA-FT,6,1.0,0.2,87.02,78.64,9.12,1.48
7,H-UNet-HPCA-FT,7,1.0,0.2,89.71,82.12,4.08,0.62
8,H-UNet-HPCA-FT,8,1.0,0.2,87.98,79.51,4.18,0.60
9,H-UNet-HPCA-FT,9,1.0,0.2,88.73,80.39,4.11,0.57


Dice           Jaccard  \
                                                   mean       std    mean   
Model                           Inv Temp Regime                             
H-UNet-HPCA-FT                  1.0      0.2     88.163  1.412075  79.800   
H-UNet-HPCA-T-FT                1.0      0.2     88.282  1.233818  79.918   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.2     91.040  1.135018  84.205   
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.2     91.413  0.949082  84.589   
UNet                            1.0      0.2     88.174  1.283114  79.652   
UNet-Pseudolabeling             1.0      0.2     91.079  0.644661  84.208   

                                                          Hausdorff Distance  \
                                                      std               mean   
Model                           Inv Temp Regime                                
H-UNet-HPCA-FT                  1.0      0.2     1.904766              6.267   
H-UNet-HPCA-T-FT                1.0      0.2     1.852631              4.621   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.2     1.610261              3.784   
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.2     1.361979              3.649   
UNet                            1.0      0.2     1.884173              3.743   
UNet-Pseudolabeling             1.0      0.2     0.937073              3.872   

                                                           \
                                                      std   
Model                           Inv Temp Regime             
H-UNet-HPCA-FT                  1.0      0.2     3.514880   
H-UNet-HPCA-T-FT                1.0      0.2     2.027760   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.2     1.946257   
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.2     1.953393   
UNet                            1.0      0.2     1.538253   
UNet-Pseudolabeling             1.0      0.2     1.835791   

                                                Average Surface Distance  \
                                                                    mean   
Model                           Inv Temp Regime                            
H-UNet-HPCA-FT                  1.0      0.2                       0.943   
H-UNet-HPCA-T-FT                1.0      0.2                       0.705   
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.2                       0.589   
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.2                       0.508   
UNet                            1.0      0.2                       0.578   
UNet-Pseudolabeling             1.0      0.2                       0.556   

                                                           
                                                      std  
Model                           Inv Temp Regime            
H-UNet-HPCA-FT                  1.0      0.2     0.566118  
H-UNet-HPCA-T-FT                1.0      0.2     0.336130  
H-UNet-Pseudolabeling-HPCA-FT   1.0      0.2     0.359860  
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.2     0.315834  
UNet                            1.0      0.2     0.283110  
UNet-Pseudolabeling             1.0      0.2     0.362589

<h1>KvasirSEG Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [ ]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet_base-swta_t')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

In [ ]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet_base-swta_t_ft')),
    'H-UNet-Pseudolabeling-SWTA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet-swta_t')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet-swta_t_ft')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

<h2>Evaluation - Hebbian Unsupervised Pretraining</h2>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [ ]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

INV_TEMP_KVASIR=20          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_KVASIR)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_KVASIR)).glob('run-*')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

In [ ]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

REGIMES = ['0.05', '0.1' '0.2', '0.25']       # regimes to be considered
INV_TEMP_KVASIR=10          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=kvasirSEG/unet_base/inv_temp-1').glob('regime-*')),
    'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=kvasirSEG/unet/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-hpca_t_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)